## this code used to send thingsboard data to jena fuseki server

trial sending the RDF data to apache server

In [ ]:
from rdflib import Graph, URIRef, Literal
import requests

# Set the Fuseki server endpoint
fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"

# Create an RDF graph
g = Graph()

# Add your RDF triples using g.add()
subject = URIRef("http://abacwsbuilding.cardiff.ac.uk/abacws/airq5.01")
predicate = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
object_value = URIRef("https://brickschema.org/schema/Brick/Sensor")

g.add((subject, predicate, object_value))

# Serialize the RDF data in Turtle format
rdf_data = g.serialize(format="turtle")
# Set the credentials for basic authentication
username = "admin"
password = "Suhas@551993"
# Set the headers for the request
headers = {
    "Content-Type": "text/turtle",
}

# Send the POST request to add RDF data
response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

# Check the response status
if response.status_code == 200:
    print("RDF data added successfully!")
else:
    print(f"Error adding RDF data. Status code: {response.status_code}")
    print(response.text)


verify that data is added successfully

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql_query = f"""
PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?type
WHERE {{
  <http://abacwsbuilding.cardiff.ac.uk/abacws/airq5.01> rdf:type ?type .
}}
"""
endpoint_url= "http://localhost:3030/abacws-sensor-network/sparql"
def execute_sparql_query(sparql_query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat('json')
    results = sparql.query().convert()
    return results
results = execute_sparql_query(sparql_query, endpoint_url)
results

1. GET TOKEN FROM THINGSBOARD

In [4]:
import requests

url = "http://192.168.1.85:8080/api/auth/login"     #home ip
# url = "http://10.10.249.130:8080/api/auth/login"  #university ip
headers = {"Content-Type": "application/json"}

payload = {
    "username": "suhasdevmanemail@gmail.com",
    "password": "Suhas@551993"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    # Authentication successful
    token = response.json().get("token")
    print("JWT_Token:", token)
else:
    print("Authentication failed. Status Code:", response.status_code)
    print("Response:", response.text)


JWT_Token: eyJhbGciOiJIUzUxMiJ9.eyJzdWIiOiJzdWhhc2Rldm1hbmVtYWlsQGdtYWlsLmNvbSIsInVzZXJJZCI6ImZkNDE4NDMwLTZmMjAtMTFlZS1hY2I5LWNmNGU4NGNlZjYzZCIsInNjb3BlcyI6WyJURU5BTlRfQURNSU4iXSwic2Vzc2lvbklkIjoiNGI5ZDFmZDItNDU2Yy00YWM5LThkODItNjE0YTk3MWQ1YjEwIiwiaXNzIjoidGhpbmdzYm9hcmQuaW8iLCJpYXQiOjE3MDA3ODM5MzEsImV4cCI6MTcwMTM4ODczMSwiZmlyc3ROYW1lIjoic3VoYXMiLCJsYXN0TmFtZSI6ImRldm1hbmUiLCJlbmFibGVkIjp0cnVlLCJpc1B1YmxpYyI6ZmFsc2UsInRlbmFudElkIjoiYmRlNTZjNzAtNmYyMC0xMWVlLWFjYjktY2Y0ZTg0Y2VmNjNkIiwiY3VzdG9tZXJJZCI6IjEzODE0MDAwLTFkZDItMTFiMi04MDgwLTgwODA4MDgwODA4MCJ9.g2Df1gcCgS69bwraFsOJxA89vg4tV4wnJFl0Za8MRO8kahJwcSitqStR3D0fPOlZQVwEllvwXla6uk2T227PUA


2. FETCH Localhhost TB DATA IN REAL-TIME (for TestBed Floor 5)

In [4]:
import requests
import json

device_id = '349ecb90-6f21-11ee-acb9-cf4e84cef63d'
access_token = token  
keys_to_fetch = [
    "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P","Q","R","S","T","U","V","W","X","Y","Z","A1","B1","C1","D1","E1","F1","G1","H1","I1","J1","K1","L1"
]

keys_to_replace = [
    "memsvalue", "hchoRsValue", "hchoppmvalue", "airQualityValue", "airQualitylevel", 
    "lux", "PM1Standard", "PM25Standard", "PM10Standard", 
    "PM1Atmospheric", "PM25Atmospheric", "PM10Atmospheric", "pirvalue", 
    "voltage2", "mq2rsratio", "rsroratio2", "voltage3", "mq3rsratio", 
    "rsroratio3", "voltage5", "mq5rsratio", "rsroratio5", 
    "voltage9", "mq9rsratio", "rsroratio9", "o2per", 
    "NO2", "c2h5ch", "VOC", "co_gas", "co2value", "tempreture", "humidity"
]


url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"
# url = f"http://10.10.249.130:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"

headers = {
    'Content-Type': 'application/json',
    'X-Authorization': f'Bearer {access_token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}
    print("Received data with replaced keys:")
    print(json.dumps(replaced_data, indent=2))
else:
    print(f"Failed to get data. Status Code: {response.status_code}")
    print(response.text)

Received data with replaced keys:
{
  "memsvalue": [
    {
      "ts": 1700184154093,
      "value": "282"
    }
  ],
  "hchoRsValue": [
    {
      "ts": 1700184154093,
      "value": "2.37"
    }
  ],
  "hchoppmvalue": [
    {
      "ts": 1700184154093,
      "value": "386.95"
    }
  ],
  "airQualityValue": [
    {
      "ts": 1700184154093,
      "value": "9"
    }
  ],
  "airQualitylevel": [
    {
      "ts": 1700184154093,
      "value": "1"
    }
  ],
  "lux": [
    {
      "ts": 1700184154093,
      "value": "0"
    }
  ],
  "PM1Standard": [
    {
      "ts": 1700184155175,
      "value": "40"
    }
  ],
  "PM25Standard": [
    {
      "ts": 1700184155175,
      "value": "56"
    }
  ],
  "PM10Standard": [
    {
      "ts": 1700184155175,
      "value": "66"
    }
  ],
  "PM1Atmospheric": [
    {
      "ts": 1700184155175,
      "value": "31"
    }
  ],
  "PM25Atmospheric": [
    {
      "ts": 1700184155175,
      "value": "46"
    }
  ],
  "PM10Atmospheric": [
    {
      "ts"

3. Fetch data, replace with actual keys and sort keys to send to apache jena fuseki

In [7]:
import requests
import json

device_id = '478abf30-7db2-11ee-b0f3-69bd975277c1'
# Replace 501 with the variable n
n = 5.01

access_token = token  
keys_to_fetch = [
    "airQualityValue",
    "c2h5ch",
    "co_gas",
    "co2value",
    "PM10Atmospheric",
    "hchoppmvalue",
    "humidity",
    "lux",
    "rsroratio5",
    "rsroratio9",
    "rsroratio3",
    "rsroratio2",
    "NO2",
    "o2per",
    "memsvalue",
    "tempreture",
    "VOC"
]


keys_to_send = {
    "airQualityValue": f"airq{n}",
    "c2h5ch": f"c2h5ch{n}",
    "co_gas": f"co{n}",
    "co2value": f"cotwo{n}",
    "PM10Atmospheric": f"dust{n}",
    "hchoppmvalue": f"hcho{n}",
    "humidity": f"hum{n}",
    "lux": f"light{n}",
    "rsroratio5": f"mqfive{n}",
    "rsroratio9": f"mqnine{n}",
    "rsroratio3": f"mqthree{n}",
    "rsroratio2": f"mqtwo{n}",
    "NO2": f"notwo{n}",
    "o2per": f"oxy{n}",
    "memsvalue": f"sound{n}",
    "tempreture": f"temp{n}",
    "VOC": f"voc{n}"
}


url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"
# url = f"http://10.10.249.130:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"


headers = {
    'Content-Type': 'application/json',
    'X-Authorization': f'Bearer {access_token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    # replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}

    print("Received data with replaced keys:")
    print(json.dumps(data, indent=2))

    new_json_data = {}

    for old_key, new_key in keys_to_send.items():
        if old_key in data:
            new_json_data[new_key] = data[old_key]


    print("Final data with replaced keys:")
    print(json.dumps(new_json_data, indent=2))
else:
    print(f"Failed to get data. Status Code: {response.status_code}")
    print(response.text)



Received data with replaced keys:
{
  "airQualityValue": [
    {
      "ts": 1700784114090,
      "value": "788"
    }
  ],
  "c2h5ch": [
    {
      "ts": 1700784114090,
      "value": "291"
    }
  ],
  "co_gas": [
    {
      "ts": 1700784114090,
      "value": "181"
    }
  ],
  "co2value": [
    {
      "ts": 1700784114090,
      "value": "606"
    }
  ],
  "PM10Atmospheric": [
    {
      "ts": 1700784114090,
      "value": "12"
    }
  ],
  "hchoppmvalue": [
    {
      "ts": 1700784114090,
      "value": "0.0"
    }
  ],
  "humidity": [
    {
      "ts": 1700784114090,
      "value": "18.7"
    }
  ],
  "lux": [
    {
      "ts": 1700784114090,
      "value": "594.75"
    }
  ],
  "rsroratio5": [
    {
      "ts": 1700784114090,
      "value": "41.34"
    }
  ],
  "rsroratio9": [
    {
      "ts": 1700784114090,
      "value": "83.94"
    }
  ],
  "rsroratio3": [
    {
      "ts": 1700784114090,
      "value": "755.43"
    }
  ],
  "rsroratio2": [
    {
      "ts": 170078411409

3. For Thingsboard Localhost

4. Add time-series data into jena fuseki server

In [ ]:
print('sensors-json :', new_json_data)

In [ ]:
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# Replace with your Fuseki server endpoint
fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
# Replace with your Fuseki server username and password
username = "admin"
password = "Suhas@551993"

# # Load RDF data from the Fuseki server
g = Graph()

# # Define namespaces
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
brick = Namespace("https://brickschema.org/schema/Brick#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
sh = Namespace("http://www.w3.org/ns/shacl#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

# Sensor data
sensor_data = new_json_data

# Iterate through sensor data and update RDF graph
for sensor_name, data_list in sensor_data.items():
    for data in data_list:
        sensor_uri = bldg[sensor_name]
        timestamp = datetime.utcfromtimestamp(data["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
        value = data["value"]

        # Update RDF graph
        g.add((sensor_uri, RDF.type, brick.Sensor))
        g.add((sensor_uri, brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
        g.add((sensor_uri, brick.hasValue, Literal(value)))


# Serialize the updated RDF graph
rdf_data = g.serialize(format="turtle")
# Set the headers for the request
headers = {
    "Content-Type": "text/turtle",
}

# Send the POST request to add RDF data
response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

# Check the response status
if response.status_code == 200:
    print("RDF data added successfully!")
else:
    print(f"Error adding RDF data. Status code: {response.status_code}")
    print(response.text)

# this is different approach

In [ ]:
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# Replace with your Fuseki server endpoint
fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
# Replace with your Fuseki server username and password
username = "admin"
password = "Suhas@551993"

# Load RDF data from the Fuseki server
g = Graph()

# Define namespaces
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
brick = Namespace("https://brickschema.org/schema/Brick#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
sh = Namespace("http://www.w3.org/ns/shacl#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

# Sensor data
sensor_data = {
    'airq5.01': [{'ts': 1700184154093, 'value': '9'}],
    'c2h5ch5.01': [{'ts': 1700184185083, 'value': '720'}],
    'co5.01': [{'ts': 1700184185083, 'value': '335'}],
    'cotwo5.01': [{'ts': 1700184185083, 'value': '6664'}],
    'dust5.01': [{'ts': 1700184155175, 'value': '54'}],
    'hcho5.01': [{'ts': 1700184154093, 'value': '386.95'}],
    'hum5.01': [{'ts': 1700184185083, 'value': '65.07'}],
    'light5.01': [{'ts': 1700184154093, 'value': '0'}],
    'mqfive5.01': [{'ts': 1700184175037, 'value': '16.79'}],
    'mqnine5.01': [{'ts': 1700184175037, 'value': '23.67'}],
    'mqthree5.01': [{'ts': 1700184165267, 'value': '79.62'}],
    'mqtwo5.01': [{'ts': 1700184165267, 'value': '10.44'}],
    'notwo5.01': [{'ts': 1700184185083, 'value': '497'}],
    'oxy5.01': [{'ts': 1700263191689, 'value': None}],
    'pir5.01': [{'ts': 1700184155175, 'value': '0'}],
    'sound5.01': [{'ts': 1700184154093, 'value': '282'}],
    'temp5.01': [{'ts': 1700184185083, 'value': '25.74'}],
    'voc5.01': [{'ts': 1700184185083, 'value': '709'}]
}

# Iterate through sensor data and update RDF graph
for sensor_name, data_list in sensor_data.items():
    for data in data_list:
        # Create unique URI for each sensor reading
        sensor_reading_uri = bldg[sensor_name + "_reading_" + str(data["ts"])]

        # Create URI for sensor
        sensor_uri = bldg[sensor_name]

        timestamp = datetime.utcfromtimestamp(data["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
        value = data["value"]

        # Update RDF graph
        g.add((sensor_reading_uri, RDF.type, brick.SensorReading))
        g.add((sensor_reading_uri, brick.hasSensor, sensor_uri))
        g.add((sensor_reading_uri, brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
        g.add((sensor_reading_uri, brick.hasValue, Literal(value)))

# Serialize the updated RDF graph
rdf_data = g.serialize(format="turtle")

# Set the headers for the request
headers = {
    "Content-Type": "text/turtle",
}

# Send the POST request to add RDF data
response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

# Check the response status
if response.status_code == 200:
    print("RDF data added successfully!")
else:
    print(f"Error adding RDF data. Status code: {response.status_code}")
    print(response.text)


In [ ]:
import requests

# Define your Fuseki server endpoint and dataset name
fuseki_endpoint = 'http://localhost:3030/abacws-sensor-network/data'

# Your sensor data
sensor_data = {
    'airq5.01': [{'ts': 1700184154093, 'value': '9'}],
    'c2h5ch5.01': [{'ts': 1700184185083, 'value': '720'}],
    'co5.01': [{'ts': 1700184185083, 'value': '335'}],
    'cotwo5.01': [{'ts': 1700184185083, 'value': '6664'}],
    'dust5.01': [{'ts': 1700184155175, 'value': '54'}],
    'hcho5.01': [{'ts': 1700184154093, 'value': '386.95'}],
    'hum5.01': [{'ts': 1700184185083, 'value': '65.07'}],
    'light5.01': [{'ts': 1700184154093, 'value': '0'}],
    'mqfive5.01': [{'ts': 1700184175037, 'value': '16.79'}],
    'mqnine5.01': [{'ts': 1700184175037, 'value': '23.67'}],
    'mqthree5.01': [{'ts': 1700184165267, 'value': '79.62'}],
    'mqtwo5.01': [{'ts': 1700184165267, 'value': '10.44'}],
    'notwo5.01': [{'ts': 1700184185083, 'value': '497'}],
    'oxy5.01': [{'ts': 1700263191689, 'value': None}],
    'pir5.01': [{'ts': 1700184155175, 'value': '0'}],
    'sound5.01': [{'ts': 1700184154093, 'value': '282'}],
    'temp5.01': [{'ts': 1700184185083, 'value': '25.74'}],
    'voc5.01': [{'ts': 1700184185083, 'value': '709'}]
}

# Iterate through sensor data and construct SPARQL update queries
for sensor, readings in sensor_data.items():
    for reading in readings:
        ts = reading['ts']
        value = reading['value']

        # Construct SPARQL update query
        update_query = f"""
        PREFIX ex: <http://example.org#>
        INSERT DATA {{
            ex:{sensor} ex:timestamp {ts} ;
                        ex:value "{value}" .
        }}
        """

        # Make SPARQL update request to the Fuseki server
        response = requests.post(fuseki_endpoint, data={'update': update_query})

        # Print the response
        print(f"Inserted data for {sensor}: {response.status_code} - {response.text}")
docker ps -a

Verify

In [ ]:
import requests
from requests.auth import HTTPBasicAuth  # Import the HTTPBasicAuth class

# Define your Fuseki server endpoint and dataset name
fuseki_endpoint = 'http://localhost:3030/abacws-sensor-network/query'

# Define the sensor and timestamp you want to query
sensor_to_query = 'airq5.01'
timestamp_to_query = 1700184154093  # Replace with the desired timestamp
fuseki_username = 'admin'  # Replace with your actual username
fuseki_password = 'Suhas@551993'

# Construct SPARQL query
sparql_query = f"""
PREFIX ex: <http://example.org#>
SELECT ?value
WHERE {{
    ex:{sensor_to_query} ex:timestamp {timestamp_to_query} ;
                    ex:value ?value .
}}
"""

# Make SPARQL query request to the Fuseki server
headers = {'Content-Type': 'application/sparql-query'}
response = requests.post(fuseki_endpoint, data=sparql_query, auth=HTTPBasicAuth(fuseki_username, fuseki_password), headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    result = response.json()

    # Check if there are results
    if 'results' in result and 'bindings' in result['results']:
        bindings = result['results']['bindings']
        if len(bindings) > 0:
            # Extract the sensor value from the first result
            sensor_value = bindings[0]['value']['value']
            print(f"Sensor value for {sensor_to_query} at timestamp {timestamp_to_query}: {sensor_value}")
        else:
            print(f"No data found for {sensor_to_query} at timestamp {timestamp_to_query}")
    else:
        print("No results found in the response.")
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")


# this is old trial

verify that data is added successfully

1. try query to receive sensor type

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql_query = f"""
PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?type
WHERE {{
  bldg:notwo5.01 rdf:type ?type .
}}
"""
endpoint_url= "http://localhost:3030/abacws-sensor-network/sparql"
def execute_sparql_query(sparql_query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat('json')
    results = sparql.query().convert()
    return results
results = execute_sparql_query(sparql_query, endpoint_url)
results

2. try query to get sensor time series data

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?value ?timestamp
WHERE {
  bldg:notwo5.01 rdf:type brick:Sensor ;
                 brick:hasValue ?value ;
                 brick:hasTimestamp ?timestamp .
}

"""

endpoint_url = "http://localhost:3030/abacws-sensor-network/sparql"

def execute_sparql_query(sparql_query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

results = execute_sparql_query(sparql_query, endpoint_url)

if results:
    for result in results["results"]["bindings"]:
        value = result["value"]["value"]
        timestamp = result["timestamp"]["value"]
        print(f"Sensor Value: {value}, Timestamp: {timestamp}")
else:
    print("No results found.")


# Combining Codes

In [ ]:
import requests
import json

device_id = '349ecb90-6f21-11ee-acb9-cf4e84cef63d'
# Replace 501 with the variable n
n = 5.01

access_token = token  
keys_to_fetch = [
    "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P","Q","R","S","T","U","V","W","X","Y","Z","A1","B1","C1","D1","E1","F1","G1","H1","I1","J1","K1","L1"
]


keys_to_replace = [
    "memsvalue", "hchoRsValue", "hchoppmvalue", "airQualityValue", "airQualitylevel", 
    "lux", "PM1Standard", "PM25Standard", "PM10Standard", 
    "PM1Atmospheric", "PM25Atmospheric", "PM10Atmospheric", "pirvalue", 
    "voltage2", "mq2rsratio", "rsroratio2", "voltage3", "mq3rsratio", 
    "rsroratio3", "voltage5", "mq5rsratio", "rsroratio5", 
    "voltage9", "mq9rsratio", "rsroratio9", "o2per", 
    "NO2", "c2h5ch", "VOC", "co_gas", "co2value", "tempreture", "humidity"
]

keys_to_send = {
    "airQualityValue": f"airq{n}",
    "c2h5ch": f"c2h5ch{n}",
    "co_gas": f"co{n}",
    "co2value": f"cotwo{n}",
    "PM10Atmospheric": f"dust{n}",
    "hchoppmvalue": f"hcho{n}",
    "humidity": f"hum{n}",
    "lux": f"light{n}",
    "rsroratio5": f"mqfive{n}",
    "rsroratio9": f"mqnine{n}",
    "rsroratio3": f"mqthree{n}",
    "rsroratio2": f"mqtwo{n}",
    "NO2": f"notwo{n}",
    "o2per": f"oxy{n}",
    "pirvalue": f"pir{n}",
    "memsvalue": f"sound{n}",
    "tempreture": f"temp{n}",
    "VOC": f"voc{n}"
}


url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"

headers = {
    'Content-Type': 'application/json',
    'X-Authorization': f'Bearer {access_token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}

    print("Received data with replaced keys:")
    print(json.dumps(replaced_data, indent=2))

    new_json_data = {}

    for old_key, new_key in keys_to_send.items():
        if old_key in replaced_data:
            new_json_data[new_key] = replaced_data[old_key]


    print("Final data with replaced keys:")
    print(json.dumps(new_json_data, indent=2))
else:
    print(f"Failed to get data. Status Code: {response.status_code}")
    print(response.text)

from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# Replace with your Fuseki server endpoint
fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
# Replace with your Fuseki server username and password
username = "admin"
password = "Suhas@551993"

# # Load RDF data from the Fuseki server
g = Graph()

# # Define namespaces
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
brick = Namespace("https://brickschema.org/schema/Brick#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
sh = Namespace("http://www.w3.org/ns/shacl#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

# Sensor data
sensor_data = new_json_data

# Iterate through sensor data and update RDF graph
for sensor_name, data_list in sensor_data.items():
    for data in data_list:
        sensor_uri = bldg[sensor_name]
        timestamp = datetime.utcfromtimestamp(data["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
        value = data["value"]

        # Update RDF graph
        g.add((sensor_uri, RDF.type, brick.Sensor))
        g.add((sensor_uri, brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
        g.add((sensor_uri, brick.hasValue, Literal(value)))


# Serialize the updated RDF graph
rdf_data = g.serialize(format="turtle")
# Set the headers for the request
headers = {
    "Content-Type": "text/turtle",
}

# Send the POST request to add RDF data
response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

# Check the response status
if response.status_code == 200:
    print("RDF data added successfully!")
else:
    print(f"Error adding RDF data. Status code: {response.status_code}")
    print(response.text)



In [ ]:
 
all_devices = [
"5.01":"478abf30-7db2-11ee-b0f3-69bd975277c1",
"5.02":"5306a860-7db2-11ee-b0f3-69bd975277c1",
"5.03":"6b52c020-7db2-11ee-b0f3-69bd975277c1",
"5.04":"725a3830-7db2-11ee-b0f3-69bd975277c1",
"5.05":"78df12c0-7db2-11ee-b0f3-69bd975277c1",
"5.06":"8084c6a0-7db2-11ee-b0f3-69bd975277c1",
"5.07":"8c34df30-7db2-11ee-b0f3-69bd975277c1",
"5.08":"93834b50-7db2-11ee-b0f3-69bd975277c1",
"5.09":"997b9d50-7db2-11ee-b0f3-69bd975277c1",
"5.10":"9f44f010-7db2-11ee-b0f3-69bd975277c1",
"5.11":"a6250a00-7db2-11ee-b0f3-69bd975277c1",
"5.12":"add0b150-7db2-11ee-b0f3-69bd975277c1",
"5.13":"b4f415d0-7db2-11ee-b0f3-69bd975277c1",
"5.14":"bad9ca30-7db2-11ee-b0f3-69bd975277c1",
"5.15":"c0fe3540-7db2-11ee-b0f3-69bd975277c1",
"5.16":"729855d0-7db6-11ee-b0f3-69bd975277c1",
"5.17":"328afde0-83f5-11ee-a992-8978af4232d8",
"5.18":"37c9eaf0-83f5-11ee-a992-8978af4232d8",
"5.19":"3d369ec0-83f5-11ee-a992-8978af4232d8",
"5.20":"4e709010-83f5-11ee-a992-8978af4232d8",
"5.21":"53491a30-83f5-11ee-a992-8978af4232d8",
"5.22":"58b83f00-83f5-11ee-a992-8978af4232d8",
"5.23":"5d800040-83f5-11ee-a992-8978af4232d8",
"5.24":"620958f0-83f5-11ee-a992-8978af4232d8",
"5.25":"6694ad70-83f5-11ee-a992-8978af4232d8",
"5.26":"6b6a7870-83f5-11ee-a992-8978af4232d8",
"5.27":"7091e5e0-83f5-11ee-a992-8978af4232d8",
"5.28":"759e7850-83f5-11ee-a992-8978af4232d8",
"5.29":"7b537fc0-83f5-11ee-a992-8978af4232d8",
"5.30":"7ffd58c0-83f5-11ee-a992-8978af4232d8",
"5.31":"84bd51d0-83f5-11ee-a992-8978af4232d8",
"5.32":"8afd3420-83f5-11ee-a992-8978af4232d8",
"5.33":"90088e10-83f5-11ee-a992-8978af4232d8",
"5.34":"9586a700-83f5-11ee-a992-8978af4232d8"
]

# FINAL CODE WITH Iteration for all devices.

## 1. For Testbed


In [ ]:
import requests
import json
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
import time  

def send_timeseries_to_fuseki():
    all_devices = {
    "5.01":"478abf30-7db2-11ee-b0f3-69bd975277c1",
    "5.02":"5306a860-7db2-11ee-b0f3-69bd975277c1",
    "5.03":"6b52c020-7db2-11ee-b0f3-69bd975277c1",
    "5.04":"725a3830-7db2-11ee-b0f3-69bd975277c1",
    "5.05":"78df12c0-7db2-11ee-b0f3-69bd975277c1",
    "5.06":"8084c6a0-7db2-11ee-b0f3-69bd975277c1",
    "5.07":"8c34df30-7db2-11ee-b0f3-69bd975277c1",
    "5.08":"93834b50-7db2-11ee-b0f3-69bd975277c1",
    "5.09":"997b9d50-7db2-11ee-b0f3-69bd975277c1",
    "5.10":"9f44f010-7db2-11ee-b0f3-69bd975277c1",
    "5.11":"a6250a00-7db2-11ee-b0f3-69bd975277c1",
    "5.12":"add0b150-7db2-11ee-b0f3-69bd975277c1",
    "5.13":"b4f415d0-7db2-11ee-b0f3-69bd975277c1",
    "5.14":"bad9ca30-7db2-11ee-b0f3-69bd975277c1",
    "5.15":"c0fe3540-7db2-11ee-b0f3-69bd975277c1",
    "5.16":"729855d0-7db6-11ee-b0f3-69bd975277c1",
    "5.17":"328afde0-83f5-11ee-a992-8978af4232d8",
    "5.18":"37c9eaf0-83f5-11ee-a992-8978af4232d8",
    "5.19":"3d369ec0-83f5-11ee-a992-8978af4232d8",
    "5.20":"4e709010-83f5-11ee-a992-8978af4232d8",
    "5.21":"53491a30-83f5-11ee-a992-8978af4232d8",
    "5.22":"58b83f00-83f5-11ee-a992-8978af4232d8",
    "5.23":"5d800040-83f5-11ee-a992-8978af4232d8",
    "5.24":"620958f0-83f5-11ee-a992-8978af4232d8",
    "5.25":"6694ad70-83f5-11ee-a992-8978af4232d8",
    "5.26":"6b6a7870-83f5-11ee-a992-8978af4232d8",
    "5.27":"7091e5e0-83f5-11ee-a992-8978af4232d8",
    "5.28":"759e7850-83f5-11ee-a992-8978af4232d8",
    "5.29":"7b537fc0-83f5-11ee-a992-8978af4232d8",
    "5.30":"7ffd58c0-83f5-11ee-a992-8978af4232d8",
    "5.31":"84bd51d0-83f5-11ee-a992-8978af4232d8",
    "5.32":"8afd3420-83f5-11ee-a992-8978af4232d8",
    "5.33":"90088e10-83f5-11ee-a992-8978af4232d8",
    "5.34":"9586a700-83f5-11ee-a992-8978af4232d8"
    }

    keys_to_fetch = [
        "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P","Q","R","S","T","U","V","W","X","Y","Z","A1","B1","C1","D1","E1","F1","G1","H1","I1","J1","K1","L1"
    ]


    keys_to_replace = [
        "memsvalue", "hchoRsValue", "hchoppmvalue", "airQualityValue", "airQualitylevel", 
        "lux", "PM1Standard", "PM25Standard", "PM10Standard", 
        "PM1Atmospheric", "PM25Atmospheric", "PM10Atmospheric", "pirvalue", 
        "voltage2", "mq2rsratio", "rsroratio2", "voltage3", "mq3rsratio", 
        "rsroratio3", "voltage5", "mq5rsratio", "rsroratio5", 
        "voltage9", "mq9rsratio", "rsroratio9", "o2per", 
        "NO2", "c2h5ch", "VOC", "co_gas", "co2value", "tempreture", "humidity"
    ]



    # Iterate over all devices
    for n, device_id in all_devices.items():
        # Your existing code here, replacing the fixed device_id and n with the current values
        device_id = device_id
        n = float(n)  # Convert n to float if needed

        # step 1. get access token

        url1 = "http://192.168.1.85:8080/api/auth/login"
        headers1 = {"Content-Type": "application/json"}

        payload1 = {
            "username": "suhasdevmanemail@gmail.com",
            "password": "Suhas@551993"
        }

        response = requests.post(url1, json=payload1, headers=headers1)

        if response.status_code == 200:
            # Authentication successful
            token = response.json().get("token")
            print("JWT_Token:", token)
        else:
            print("Authentication failed. Status Code:", response.status_code)
            print("Response:", response.text)

        
        access_token = token  
        # step 2. 

        url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"

        headers = {
            'Content-Type': 'application/json',
            'X-Authorization': f'Bearer {access_token}'
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}

            print("Received data with replaced keys:")
            print(json.dumps(replaced_data, indent=2))

            keys_to_send = {
                "airQualityValue": f"airq{n}",
                "c2h5ch": f"c2h5ch{n}",
                "co_gas": f"co{n}",
                "co2value": f"cotwo{n}",
                "PM10Atmospheric": f"dust{n}",
                "hchoppmvalue": f"hcho{n}",
                "humidity": f"hum{n}",
                "lux": f"light{n}",
                "rsroratio5": f"mqfive{n}",
                "rsroratio9": f"mqnine{n}",
                "rsroratio3": f"mqthree{n}",
                "rsroratio2": f"mqtwo{n}",
                "NO2": f"notwo{n}",
                "o2per": f"oxy{n}",
                "pirvalue": f"pir{n}",
                "memsvalue": f"sound{n}",
                "tempreture": f"temp{n}",
                "VOC": f"voc{n}"
            }


            # prepair data to send apache jena fuseki server
            new_json_data = {}

            for old_key, new_key in keys_to_send.items():
                if old_key in replaced_data:
                    new_json_data[new_key] = replaced_data[old_key]


            print("Final data with replaced keys:")
            print(json.dumps(new_json_data, indent=2))
        else:
            print(f"Failed to get data. Status Code: {response.status_code}")
            print(response.text)



        # Replace with your Fuseki server endpoint
        fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
        # Replace with your Fuseki server username and password
        username = "admin"
        password = "Suhas@551993"

        # # Load RDF data from the Fuseki server
        g = Graph()

        # # Define namespaces
        bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
        brick = Namespace("https://brickschema.org/schema/Brick#")
        owl = Namespace("http://www.w3.org/2002/07/owl#")
        rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
        rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
        sh = Namespace("http://www.w3.org/ns/shacl#")
        xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

        # Sensor data
        sensor_data = new_json_data

        # Iterate through sensor data and update RDF graph
        for sensor_name, data_list in sensor_data.items():
            for data in data_list:
                sensor_uri = bldg[sensor_name]
                timestamp = datetime.utcfromtimestamp(data["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
                value = data["value"]

                # Update RDF graph
                g.add((sensor_uri, RDF.type, brick.Sensor))
                g.add((sensor_uri, brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
                g.add((sensor_uri, brick.hasValue, Literal(value)))

        # Serialize the updated RDF graph
        rdf_data = g.serialize(format="turtle")
        # Set the headers for the request
        headers = {
            "Content-Type": "text/turtle",
        }

        # Send the POST request to add RDF data
        response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

        # Check the response status
        if response.status_code == 200:
            print("RDF data added successfully!")
        else:
            print(f"Error adding RDF data. Status code: {response.status_code}")
            print(response.text)

# Run the code every 5 seconds
while True:
    send_timeseries_to_fuseki()
    time.sleep(5)

# New trial with UUID

1. single sensor

In [3]:
import requests

url = "http://192.168.1.85:8080/api/auth/login"     #home ip
# url = "http://10.10.249.130:8080/api/auth/login"  #university ip
headers = {"Content-Type": "application/json"}

payload = {
    "username": "suhasdevmanemail@gmail.com",
    "password": "Suhas@551993"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    # Authentication successful
    token = response.json().get("token")
    print("JWT_Token:", token)
else:
    print("Authentication failed. Status Code:", response.status_code)
    print("Response:", response.text)


JWT_Token: eyJhbGciOiJIUzUxMiJ9.eyJzdWIiOiJzdWhhc2Rldm1hbmVtYWlsQGdtYWlsLmNvbSIsInVzZXJJZCI6ImZkNDE4NDMwLTZmMjAtMTFlZS1hY2I5LWNmNGU4NGNlZjYzZCIsInNjb3BlcyI6WyJURU5BTlRfQURNSU4iXSwic2Vzc2lvbklkIjoiZTM0ZDEwMTgtNjFkZi00MjBkLTk4MzAtMzkyNDY1OTRkMzVmIiwiaXNzIjoidGhpbmdzYm9hcmQuaW8iLCJpYXQiOjE3MDA3Nzc1ODQsImV4cCI6MTcwMTM4MjM4NCwiZmlyc3ROYW1lIjoic3VoYXMiLCJsYXN0TmFtZSI6ImRldm1hbmUiLCJlbmFibGVkIjp0cnVlLCJpc1B1YmxpYyI6ZmFsc2UsInRlbmFudElkIjoiYmRlNTZjNzAtNmYyMC0xMWVlLWFjYjktY2Y0ZTg0Y2VmNjNkIiwiY3VzdG9tZXJJZCI6IjEzODE0MDAwLTFkZDItMTFiMi04MDgwLTgwODA4MDgwODA4MCJ9.o3e3ceR5i3ukIYklBoLPaS2F9CO2BsybEsN-lKHdh27ZDNvv8wxmB4sjpaHYgnWi4Z820l5v8h-Tas5VQsj2mQ


In [6]:
import requests
import json

device_id = '478abf30-7db2-11ee-b0f3-69bd975277c1'
# Replace 501 with the variable n
n = 5.01

access_token = token  
keys_to_fetch = [
    "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P","Q","R","S","T","U","V","W","X","Y","Z","A1","B1","C1","D1","E1","F1","G1","H1","I1","J1","K1","L1"
]


keys_to_replace = [
    "memsvalue", "hchoRsValue", "hchoppmvalue", "airQualityValue", "airQualitylevel", 
    "lux", "PM1Standard", "PM25Standard", "PM10Standard", 
    "PM1Atmospheric", "PM25Atmospheric", "PM10Atmospheric", "pirvalue", 
    "voltage2", "mq2rsratio", "rsroratio2", "voltage3", "mq3rsratio", 
    "rsroratio3", "voltage5", "mq5rsratio", "rsroratio5", 
    "voltage9", "mq9rsratio", "rsroratio9", "o2per", 
    "NO2", "c2h5ch", "VOC", "co_gas", "co2value", "tempreture", "humidity"
]

keys_to_send = {
    "airQualityValue": f"airq{n}",
    "c2h5ch": f"c2h5ch{n}",
    "co_gas": f"co{n}",
    "co2value": f"cotwo{n}",
    "PM10Atmospheric": f"dust{n}",
    "hchoppmvalue": f"hcho{n}",
    "humidity": f"hum{n}",
    "lux": f"light{n}",
    "rsroratio5": f"mqfive{n}",
    "rsroratio9": f"mqnine{n}",
    "rsroratio3": f"mqthree{n}",
    "rsroratio2": f"mqtwo{n}",
    "NO2": f"notwo{n}",
    "o2per": f"oxy{n}",
    "pirvalue": f"pir{n}",
    "memsvalue": f"sound{n}",
    "tempreture": f"temp{n}",
    "VOC": f"voc{n}"
}


url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"
# url = f"http://10.10.249.130:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"


headers = {
    'Content-Type': 'application/json',
    'X-Authorization': f'Bearer {access_token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}

    print("Received data with replaced keys:")
    print(json.dumps(replaced_data, indent=2))

    new_json_data = {}

    for old_key, new_key in keys_to_send.items():
        if old_key in replaced_data:
            new_json_data[new_key] = replaced_data[old_key]


    print("Final data with replaced keys:")
    print(json.dumps(new_json_data, indent=2))
else:
    print(f"Failed to get data. Status Code: {response.status_code}")
    print(response.text)



Received data with replaced keys:
{
  "memsvalue": [
    {
      "ts": 1700777673330,
      "value": null
    }
  ],
  "hchoRsValue": [
    {
      "ts": 1700777673331,
      "value": null
    }
  ],
  "hchoppmvalue": [
    {
      "ts": 1700777673331,
      "value": null
    }
  ],
  "airQualityValue": [
    {
      "ts": 1700777673332,
      "value": null
    }
  ],
  "airQualitylevel": [
    {
      "ts": 1700777673332,
      "value": null
    }
  ],
  "lux": [
    {
      "ts": 1700777673333,
      "value": null
    }
  ],
  "PM1Standard": [
    {
      "ts": 1700777673334,
      "value": null
    }
  ],
  "PM25Standard": [
    {
      "ts": 1700777673334,
      "value": null
    }
  ],
  "PM10Standard": [
    {
      "ts": 1700777673335,
      "value": null
    }
  ],
  "PM1Atmospheric": [
    {
      "ts": 1700777673335,
      "value": null
    }
  ],
  "PM25Atmospheric": [
    {
      "ts": 1700777673335,
      "value": null
    }
  ],
  "PM10Atmospheric": [
    {
      "ts": 17

In [3]:
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, XSD
from datetime import datetime
import uuid
import requests

# Replace with your Fuseki server endpoint
fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
# Replace with your Fuseki server username and password
username = "admin"
password = "Suhas@551993"

# Load RDF data from the Fuseki server
g = Graph()

# Define namespaces
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
brick = Namespace("https://brickschema.org/schema/Brick#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
sh = Namespace("http://www.w3.org/ns/shacl#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

def generate_uuid():
    return str(uuid.uuid4())

# Sensor data
sensor_data = new_json_data

# Iterate through sensor data and update RDF graph
for sensor_name, data_list in sensor_data.items():
    for data in data_list:
        sensor_uri = bldg[sensor_name]
        timestamp = datetime.utcfromtimestamp(data["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
        value = data["value"]
        data_uuid = generate_uuid()

        # Update RDF graph
        g.add((bldg[data_uuid], RDF.type, brick.SensorData))
        g.add((bldg[data_uuid], brick.hasSensor, sensor_uri))
        g.add((bldg[data_uuid], brick.hasUUID, Literal(data_uuid, datatype=XSD.string)))
        g.add((bldg[data_uuid], brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
        g.add((bldg[data_uuid], brick.hasValue, Literal(value)))

# Serialize the updated RDF graph
rdf_data = g.serialize(format="turtle")

# Set the headers for the request
headers = {
    "Content-Type": "text/turtle",
}

# Send the POST request to add RDF data
response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

# Check the response status
if response.status_code == 200:
    print("RDF data added successfully!")
else:
    print(f"Error adding RDF data. Status code: {response.status_code}")
    print(response.text)
    


ConnectionError: HTTPConnectionPool(host='localhost', port=3030): Max retries exceeded with url: /abacws-sensor-network/data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4d9505bdf0>: Failed to establish a new connection: [Errno 111] Connection refused'))

finl trial code with uuid

In [ ]:
import requests
import json
from rdflib import Graph, Literal, Namespace
from rdflib.namespace import RDF, XSD
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import uuid

def generate_uuid():
    return str(uuid.uuid4())

def send_timeseries_to_fuseki():
    all_devices = {
    "5.01":"478abf30-7db2-11ee-b0f3-69bd975277c1",
    "5.02":"5306a860-7db2-11ee-b0f3-69bd975277c1",
    "5.03":"6b52c020-7db2-11ee-b0f3-69bd975277c1",
    "5.04":"725a3830-7db2-11ee-b0f3-69bd975277c1",
    "5.05":"78df12c0-7db2-11ee-b0f3-69bd975277c1",
    "5.06":"8084c6a0-7db2-11ee-b0f3-69bd975277c1",
    "5.07":"8c34df30-7db2-11ee-b0f3-69bd975277c1",
    "5.08":"93834b50-7db2-11ee-b0f3-69bd975277c1",
    "5.09":"997b9d50-7db2-11ee-b0f3-69bd975277c1",
    "5.10":"9f44f010-7db2-11ee-b0f3-69bd975277c1",
    "5.11":"a6250a00-7db2-11ee-b0f3-69bd975277c1",
    "5.12":"add0b150-7db2-11ee-b0f3-69bd975277c1",
    "5.13":"b4f415d0-7db2-11ee-b0f3-69bd975277c1",
    "5.14":"bad9ca30-7db2-11ee-b0f3-69bd975277c1",
    "5.15":"c0fe3540-7db2-11ee-b0f3-69bd975277c1",
    "5.16":"729855d0-7db6-11ee-b0f3-69bd975277c1",
    "5.17":"328afde0-83f5-11ee-a992-8978af4232d8",
    "5.18":"37c9eaf0-83f5-11ee-a992-8978af4232d8",
    "5.19":"3d369ec0-83f5-11ee-a992-8978af4232d8",
    "5.20":"4e709010-83f5-11ee-a992-8978af4232d8",
    "5.21":"53491a30-83f5-11ee-a992-8978af4232d8",
    "5.22":"58b83f00-83f5-11ee-a992-8978af4232d8",
    "5.23":"5d800040-83f5-11ee-a992-8978af4232d8",
    "5.24":"620958f0-83f5-11ee-a992-8978af4232d8",
    "5.25":"6694ad70-83f5-11ee-a992-8978af4232d8",
    "5.26":"6b6a7870-83f5-11ee-a992-8978af4232d8",
    "5.27":"7091e5e0-83f5-11ee-a992-8978af4232d8",
    "5.28":"759e7850-83f5-11ee-a992-8978af4232d8",
    "5.29":"7b537fc0-83f5-11ee-a992-8978af4232d8",
    "5.30":"7ffd58c0-83f5-11ee-a992-8978af4232d8",
    "5.31":"84bd51d0-83f5-11ee-a992-8978af4232d8",
    "5.32":"8afd3420-83f5-11ee-a992-8978af4232d8",
    "5.33":"90088e10-83f5-11ee-a992-8978af4232d8",
    "5.34":"9586a700-83f5-11ee-a992-8978af4232d8"
    }

    keys_to_fetch = [
        "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
        "A1", "B1", "C1", "D1", "E1", "F1", "G1", "H1", "I1", "J1", "K1", "L1"
    ]

    keys_to_replace = [
        "memsvalue", "hchoRsValue", "hchoppmvalue", "airQualityValue", "airQualitylevel",
        "lux", "PM1Standard", "PM25Standard", "PM10Standard",
        "PM1Atmospheric", "PM25Atmospheric", "PM10Atmospheric", "pirvalue",
        "voltage2", "mq2rsratio", "rsroratio2", "voltage3", "mq3rsratio",
        "rsroratio3", "voltage5", "mq5rsratio", "rsroratio5",
        "voltage9", "mq9rsratio", "rsroratio9", "o2per",
        "NO2", "c2h5ch", "VOC", "co_gas", "co2value", "tempreture", "humidity"
    ]

    # Iterate over all devices
    for n, device_id in all_devices.items():
        device_id = device_id
        n = float(n)  # Convert n to float if needed

        # step 1. get access token
        url1 = "http://192.168.1.85:8080/api/auth/login"
        headers1 = {"Content-Type": "application/json"}

        payload1 = {
            "username": "suhasdevmanemail@gmail.com",
            "password": "Suhas@551993"
        }

        response = requests.post(url1, json=payload1, headers=headers1)

        if response.status_code == 200:
            # Authentication successful
            token = response.json().get("token")
            print("JWT_Token:", token)
        else:
            print("Authentication failed. Status Code:", response.status_code)
            print("Response:", response.text)

        access_token = token
        # step 2.
        url = f"http://192.168.1.85:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={','.join(keys_to_fetch)}"

        headers = {
            'Content-Type': 'application/json',
            'X-Authorization': f'Bearer {access_token}'
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            replaced_data = {keys_to_replace[i]: data[key] for i, key in enumerate(keys_to_fetch) if key in data}

            print("Received data with replaced keys:")
            print(json.dumps(replaced_data, indent=2))

            keys_to_send = {
                "airQualityValue": f"airq{n}",
                "c2h5ch": f"c2h5ch{n}",
                "co_gas": f"co{n}",
                "co2value": f"cotwo{n}",
                "PM10Atmospheric": f"dust{n}",
                "hchoppmvalue": f"hcho{n}",
                "humidity": f"hum{n}",
                "lux": f"light{n}",
                "rsroratio5": f"mqfive{n}",
                "rsroratio9": f"mqnine{n}",
                "rsroratio3": f"mqthree{n}",
                "rsroratio2": f"mqtwo{n}",
                "NO2": f"notwo{n}",
                "o2per": f"oxy{n}",
                "pirvalue": f"pir{n}",
                "memsvalue": f"sound{n}",
                "tempreture": f"temp{n}",
                "VOC": f"voc{n}"
            }

            # prepare data to send to Apache Jena Fuseki server
            new_json_data = []

            for old_key, new_key in keys_to_send.items():
                if old_key in replaced_data:
                    for item in replaced_data[old_key]:
                        data_uuid = generate_uuid()
                        new_item = {
                            "uuid": data_uuid,
                            "ts": item["ts"],
                            "value": item["value"]
                        }
                        new_json_data.append({new_key: [new_item]})

            print("Final data with replaced keys and UUIDs:")
            print(json.dumps(new_json_data, indent=2))
        else:
            print(f"Failed to get data. Status Code: {response.status_code}")
            print(response.text)

        # Replace with your Fuseki server endpoint
        fuseki_endpoint = "http://localhost:3030/abacws-sensor-network/data"
        # Replace with your Fuseki server username and password
        username = "admin"
        password = "Suhas@551993"

        # Load RDF data from the Fuseki server
        g = Graph()

        # Define namespaces
        bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
        brick = Namespace("https://brickschema.org/schema/Brick#")
        owl = Namespace("http://www.w3.org/2002/07/owl#")
        rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
        rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
        sh = Namespace("http://www.w3.org/ns/shacl#")
        xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

        # Sensor data
        sensor_data = new_json_data

        # Iterate through sensor data and update RDF graph
        for data_item in sensor_data:
            for key, items in data_item.items():
                for item in items:
                    sensor_uri = bldg[key]
                    timestamp = datetime.utcfromtimestamp(item["ts"] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
                    value = item["value"]
                    data_uuid = item["uuid"]

                    # Update RDF graph
                    g.add((bldg[data_uuid], RDF.type, brick.SensorData))
                    g.add((bldg[data_uuid], brick.hasSensor, sensor_uri))
                    g.add((bldg[data_uuid], brick.hasUUID, Literal(data_uuid, datatype=XSD.string)))
                    g.add((bldg[data_uuid], brick.hasTimestamp, Literal(timestamp, datatype=xsd.dateTime)))
                    g.add((bldg[data_uuid], brick.hasValue, Literal(value)))

        # Serialize the updated RDF graph
        rdf_data = g.serialize(format="turtle")
        # Set the headers for the request
        headers = {
            "Content-Type": "text/turtle",
        }

        # Send the POST request to add RDF data
        response = requests.post(fuseki_endpoint, data=rdf_data, headers=headers, auth=(username, password))

        # Check the response status
        if response.status_code == 200:
            print("RDF data added successfully!")
        else:
            print(f"Error adding RDF data. Status code: {response.status_code}")
            print(response.text)

# Run the code every 5 seconds
while True:
    send_timeseries_to_fuseki()
    time.sleep(5)
